## Lung and Colon Histopathology Classification


### 1. Setup


In [1]:
import os
import sys
import numpy as np
from contextlib import ExitStack
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import (
    Sequential,
    layers,
    optimizers,
    regularizers,
    callbacks,
    metrics,
    losses,
    activations,
)
import tensorflow_hub as hub
from tensorflow.train import Example, Feature, Features, BytesList, Int64List, FloatList

# Local Libraries
import ml_datasets
import ml_learning_rate
import ml_plotting
import ml_functions

/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/mmenendezg/Developer/Books/.venv/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ["dlopen(/Users/mmenendezg/

In [2]:
BATCH_SIZE = 8
AUTOTUNE = tf.data.AUTOTUNE
SHUFFLE_BUFFER = 1000
DATASET_PATH = tf.io.gfile.join(os.environ["DATA_PATH"], "lung_colon_histopathology")
RAW_DATASET_PATH = tf.io.gfile.join(DATASET_PATH, "raw_data")
TFRECORDS_PATH = tf.io.gfile.join(DATASET_PATH, "tfrecord_data")
MODEL_PATH = os.path.join("..", "..", "models", "chapter_14")
SEED = 1992

# Determine the organ the sample is from
ORGAN_CLASSES = {
    0: "Colon",
    1: "Lung",
}

# Determine if the tissue is benign, adenocarcinoma or scamous cell carcinoma
TYPE_TISSUE = {
    0: "Benign",
    1: "Adenocarcinoma",
    2: "Squamous Cell Carcinoma",
}

IMG_SIZE = (768, 768)
IMG_CHANNELS = 3
NEW_IMG_SIZE = [512, 512, 3]

# Model
MODEL_URL = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"

## Notes:

- The architecture used is ResNet-18, as used and suggested in [this article](https://arxiv.org/pdf/1901.11489v1.pdf).
- They used learning rate with decay; metrics were accuracy, precision, recall and f1; and the loss function was multi-class crosstentropy
- They implemented data augmentation to increase the size of the datasets. This will be dismissed in this notebook since the data has been previously augmented.


### 2. Data Pipeline


- Dataset used is [Lung and Colon Cancer Histopathological Images | Kaggle](https://www.kaggle.com/datasets/andrewmvd/lung-and-colon-cancer-histopathological-images/code)
- Images are in folders, each folder a class
- Images size is `[256, 256, 3]`
- Dataset has been augmented using [`Augmentor`](https://augmentor.readthedocs.io/en/stable/) package


In [3]:
def get_key_from_value(dictionary, value):
    return [key for key, v in dictionary.items() if v == value][0]


def get_folders_images(filepath):
    name_folders = tf.io.gfile.listdir(filepath)
    path_folders = [tf.io.gfile.join(filepath, folder) for folder in name_folders]

    list_folders = {folder: path for folder, path in zip(name_folders, path_folders)}
    return list_folders


def create_example(image, organ_label, tissue_label):
    image_data = tf.io.serialize_tensor(image)
    feature = {
        "image": Feature(bytes_list=BytesList(value=[image_data.numpy()])),
        "organ_label": Feature(int64_list=Int64List(value=[organ_label.numpy()])),
        "tissue_label": Feature(int64_list=Int64List(value=[tissue_label.numpy()])),
    }

    return Example(features=Features(feature=feature))


def save_protobufs(dataset, type_set="train", n_shards=10):

    set_folder = tf.io.gfile.join(TFRECORDS_PATH, type_set)

    tf.io.gfile.makedirs(set_folder)
    file_paths = [tf.io.gfile.join(set_folder, filepath) for filepath in files]

    if type_set == "train":
        dataset.shuffle(SHUFFLE_BUFFER)
    files = [
        f"{type_set}.tfrecord-{shard.numpy() + 1:02d}-of-{n_shards:02d}"
        for shard in tf.range(n_shards)
    ]

    with ExitStack() as stack:
        writers = [
            stack.enter_context(tf.io.TFRecordWriter(file)) for file in file_paths
        ]
        for index, (image, organ_label, tissue_label) in dataset.enumerate():
            shard = index % n_shards
            example = create_example(image, organ_label, tissue_label)
            writers[shard].write(example.SerializeToString())
    return file_paths


def get_folders_tfrecords(type_set="train"):
    folder = tf.io.gfile.join(TFRECORDS_PATH, type_set)
    files = tf.io.gfile.listdir(folder)
    list_files = [tf.io.gfile.join(folder, filepath) for filepath in files]
    return list_files


def save_images_protobufs():

    # Verify if the folder for the TFRecords exist to process the data
    if not tf.io.gfile.exists(TFRECORDS_PATH):

        tf.io.gfile.makedirs(TFRECORDS_PATH)

        list_folders = get_folders_images(RAW_DATASET_PATH)

        # Create the dataset per folder, split the dataset into train, valid and test sets
        # Add the organ label
        # New dimension is (image, organ_label, tissue_label)
        # organ_label -> 0: Colon, 1: Lung
        # tissue_label -> 0: Benign, 1: Adenocarcinoma, 2: Squamous Cell Carcinoma
        colon_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            list_folders["colon_images"], batch_size=None, image_size=IMG_SIZE
        )
        train_colon, valid_colon, test_colon = ml_functions.balanced_split(
            colon_dataset
        )
        train_colon = train_colon.map(
            lambda image, label: (image, 0, label), num_parallel_calls=AUTOTUNE
        )
        valid_colon = valid_colon.map(
            lambda image, label: (image, 0, label), num_parallel_calls=AUTOTUNE
        )
        test_colon = test_colon.map(
            lambda image, label: (image, 0, label), num_parallel_calls=AUTOTUNE
        )

        lung_dataset = tf.keras.preprocessing.image_dataset_from_directory(
            list_folders["lung_images"], batch_size=None, image_size=IMG_SIZE
        )
        train_lung, valid_lung, test_lung = ml_functions.balanced_split(lung_dataset)
        train_lung = train_lung.map(
            lambda image, label: (image, 1, label), num_parallel_calls=AUTOTUNE
        )
        valid_lung = valid_lung.map(
            lambda image, label: (image, 1, label), num_parallel_calls=AUTOTUNE
        )
        test_lung = test_lung.map(
            lambda image, label: (image, 1, label), num_parallel_calls=AUTOTUNE
        )

        # Combines the datasets into a single one
        train_set = train_colon.concatenate(train_lung)
        valid_set = valid_colon.concatenate(valid_lung)
        test_set = test_colon.concatenate(test_lung)

        train_paths = save_protobufs(train_set, "train")
        valid_paths = save_protobufs(valid_set, "valid")
        test_paths = save_protobufs(test_set, "test")
    else:
        train_paths = get_folders_tfrecords("train")
        valid_paths = get_folders_tfrecords("valid")
        test_paths = get_folders_tfrecords("test")

    return train_paths, valid_paths, test_paths


In [4]:
train_files, valid_files, test_files = save_images_protobufs()

In [5]:
def get_record_multilabel(tfrecord):
    feature_descriptions = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "organ_label": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "tissue_label": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
    }

    example = tf.io.parse_single_example(tfrecord, feature_descriptions)
    image = tf.io.parse_tensor(example["image"], out_type=tf.float32)
    image = tf.reshape(image, shape=[IMG_SIZE[0], IMG_SIZE[1], IMG_CHANNELS])
    image = tf.image.resize(image, size=[NEW_IMG_SIZE[0], NEW_IMG_SIZE[1]])
    return image, (example["organ_label"], example["tissue_label"])


def get_record(tfrecord):
    feature_descriptions = {
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "organ_label": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        "tissue_label": tf.io.FixedLenFeature([], tf.int64, default_value=-1),
    }

    example = tf.io.parse_single_example(tfrecord, feature_descriptions)
    image = tf.io.parse_tensor(example["image"], out_type=tf.float32)
    image = tf.reshape(image, shape=[IMG_SIZE[0], IMG_SIZE[1], IMG_CHANNELS])
    image = tf.image.resize(image, size=[NEW_IMG_SIZE[0], NEW_IMG_SIZE[1]])
    return image, example["tissue_label"]


def get_dataset(file_paths, cache=True, shuffle_buffer=None, multi_label=False):
    dataset = tf.data.TFRecordDataset(file_paths, num_parallel_reads=AUTOTUNE)

    if cache:
        dataset = dataset.cache()
    if shuffle_buffer:
        dataset = dataset.shuffle(shuffle_buffer)

    if multi_label:
        dataset = (
            dataset.map(get_record_multilabel, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE)
            .prefetch(AUTOTUNE)
        )
    else:
        dataset = (
            dataset.map(get_record, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE)
            .prefetch(AUTOTUNE)
        )

    return dataset

In [6]:
train_set = get_dataset(train_files, shuffle_buffer=SHUFFLE_BUFFER)
valid_set = get_dataset(valid_files)
test_set = get_dataset(test_files)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



### 3. Data Preprocessing


In [7]:
tf.keras.backend.clear_session()
tf.random.set_seed(SEED)


In [8]:
def train_model(
    train_set,
    valid_set,
    test_set,
    epochs,
    learning_rate,
    trainable=False,
    lr_function=None,
):
    # Normalization of the data
    def normalize(image, label):
        norm_image = image / 255.0
        return (norm_image, label)
    
    train_set_normalized = train_set.map(normalize, num_parallel_calls=AUTOTUNE)
    valid_set_normalized = valid_set.map(normalize, num_parallel_calls=AUTOTUNE)
    test_set_normalized = test_set.map(normalize, num_parallel_calls=AUTOTUNE)

    # Load the ResNet model
    base_model = hub.KerasLayer(
        MODEL_URL,
        trainable=trainable,
    )

    model = Sequential([base_model, layers.Dense(3, activation="softmax")])
    model.build([None, NEW_IMG_SIZE[0], NEW_IMG_SIZE[1], NEW_IMG_SIZE[2]])

    # Compilation of the model
    optimizer_ = optimizers.Adam(learning_rate=learning_rate)
    metrics_ = ["accuracy"]
    loss_ = "sparse_categorical_crossentropy"
    
    model.compile(
        optimizer=optimizer_,
        metrics=metrics_,
        loss=loss_
    )
    
    # Callbacks
    exponential_decay_fn = ml_learning_rate.exponential_decay_with_warmup(
        lr_start=learning_rate,
        lr_max=learning_rate * 10,
        lr_min=learning_rate / 10,
    )
    lr_scheduler_cb = callbacks.LearningRateScheduler(exponential_decay_fn)
    
    folder_logs = tf.io.gfile.join(
        "..", "..", "..", "reports", "logs", "chapter_14", "lung_colon_histopathology"
    )
    logdir = ml_functions.get_logdir(path_folder=folder_logs)
    tensorboard_cb = callbacks.TensorBoard(log_dir=logdir)
    
    model_path = tf.io.gfile.join(MODEL_PATH, "lung_colon_histopathology.h5")
    model_checkpoint_cb = callbacks.ModelCheckpoint(
        filepath=model_path, save_best_only=True
    )
    
    early_stopping_cb = callbacks.EarlyStopping(patience=5)
    
    callbacks_list = [
        lr_scheduler_cb,
        tensorboard_cb,
        model_checkpoint_cb,
        early_stopping_cb
    ]
    
    # Train the model
    model.fit(
        train_set_normalized,
        validation_data=valid_set_normalized,
        epochs=epochs,
        callbacks=callbacks_list,
    )
    
    evaluation = model.evaluate(test_set, verbose=0)
    
    return model, evaluation

In [9]:
model, evaluation = train_model(
    train_set,
    valid_set,
    test_set,
    epochs=5,
    learning_rate=1e-4,
    trainable=False,
    lr_function=None
)

Epoch 1/5
   2005/Unknown - 304s 149ms/step - loss: 0.2739 - accuracy: 0.9074

: 

: 

### 4. Model Training


### 5. Model Fine-Tuning


### 6.
